# Indeed webscraper

page 1= https://nl.indeed.com/jobs?q=data+analist&start=00  <br>
page 2= https://nl.indeed.com/jobs?q=data+analist&start=10 <br>
page 3= https://nl.indeed.com/jobs?q=data+analist&start=20 <br>


**Getting the link of the job**:

`<table id="resultsBody"> --> <td id="resultsCol"> --> <a (containing href=)>` --> href == link to job

**Getting job description**:
`<div id="jobDescriptionText"> --> <p> of (<ul> --> <li>)` --> paragraphs of text


## getting indeed search page information
base url= https://nl.indeed.com/jobs?q=data+analist

In [27]:
#basic R
library(XML)
url_job_search <- 'https://nl.indeed.com/jobs?q=data+analist'

url_job_search_read <- readLines(url_job_search, 10, encoding = "UTF-8")

url_job_search_parsed <- htmlParse(url_job_search_read, encoding = "UTF-8")

In [55]:
# Rvest
library(tidyverse)
library(rvest)

url_job_search <- read_html('https://nl.indeed.com/jobs?q=data') # lees html in

job_urls <- url_job_search %>%
  # html_nodes('body')%>%
  html_nodes(xpath= '//*[@id="mosaic-zone-jobcards"]') %>% # ga naar het specifieke xpath
  html_nodes('a') %>% # ga naar a
  html_attr("href")# pak de href

In [56]:
job_urls

[1] "/pagead/clk?mo=r&ad=-6NYlbfkN0C79Zy5_7MaUmympOED4OXIn0VfuKyjVBktQiVzKpLwu9ZYivLqPbckuIrzWNrAW7UJoeBAt2Vx5h323_faPEtEoaaHpKnSX8YwgUSt0pMUH2SiQ5wUIjdyted8C5hUgAWfEjt37lxpH9awHhbZLLKU4_gU4tL40tenvdfcUVQe0ORjFLbJ1I2Lm2nl3pU9B8fus5H04WJIpYNvnFeA1OviQuy8IfVuQZG-cIRJXgZlAPspjn5SzkWdGwp7dJBMVpgTDnaUBvbnZ5NkohfXlQ8t8kw74UgivCr6JSKU-89malt1ujkucCh0uwazW4eFdxZcTxiov89jyg4JqkdvFT1vVILLyNJPRfMBuX3gqdNBlTu-wkQvlTTWNZA-f7mRGZJS0GAvJPyQyRTpWP14Wad1yJXl4KYO1epAvC1wKdqybsrPoD0DBuJE1cd75TiGinNPSXjil5OzZGNZdvTQvmaotbGwKq2MSvVY2E16Lze9suASA8G71DVveTK9tCMuR2ctUzeVTqj_OCG_N9G_vM9WdVYGlW4LYbBYLF3RGLi7PZq4A8c_VW-kK5gY2xVTKZxwgghmlwChtPNTmygX1q3YkFHXvMVqK8CA1vxcpXg_Gd7UpzQ4diQ25xu1v4678pUzMqR6s6uF5BIU51yA_m79jr7lKzIW3mqrUtnqqRIiKSu296PXGybXpSvc9dKU4_Lb6dOtDOmm9ltG_Hmj9NUs_nhFanhH1B-PglTmKPVdMXMxBTkqv_-s9MDQ1M9sQsUJgaPUXfLkY1vwFVHJNrqFX8B9srmPYxMgQR8hRbQlm4UWX0P7KnIZ4W-kcceRyiY=&p=0&fvj=0&vjs=3"
 [2] "/company/EnAppSys-BV/jobs/Data-Scientist-5dc16bcdd3d26a33?fccid=45b7c770a2842d74&vjs=3"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
 [3] "/Enappsys-Bv-vacatures"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
 [4] "/vacatures-in-Terneuzen"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
 [5] "/career/data-scientist/salaries/Postcode-4531-in-Terneuzen?campaignid=serp-more&fromjk=5dc16bcdd3d26a33&from=serp-more"                                                                                                                                                                                                                                                                                                                                    

## getting job page info

base_url= https://nl.indeed.com

In [41]:
job_base_url <- 'https://nl.indeed.com'
full_job_url <- paste(job_base_url, job_urls[[1]], sep="")
full_job_url

ERROR: Error in job_urls[[1]]: subscript out of bounds


In [4]:
job_description <- read_html(full_job_url) %>%
    html_nodes(xpath= '//*[@id="jobDescriptionText"]') %>%
    html_text()

In [5]:
test_url = 'https://nl.indeed.com/viewjob?jk=33d4be78835757d1&tk=1ft2v35nf3b5m000&from=serp&vjs=3&advn=4610837443530944&adid=371890778&ad=-6NYlbfkN0A_cWh5ufxJCEatf9FOtgdwl-LajcKrnY3NwdklpeSDnjaH9EsBM6ngV0apa0_Kx2a5GKCW4uS0O7MdMAqsdKUS4q0L_C77bzQCZHbh2VakwHOoBSx_gXNPBJOeqxq7IUUc4s9CBf_lt5ktVCCKOckMPI8DQ-kKHnVfI97NdL-7of8Oo7OCSaCBZL8Z41xkIGi-jjU-emgJ8f1utRtU276rmh-6Rww5rOQKaWr9wwgGy6sYrcY4MEIcYBkbm5BychuYGL0AFbd7Vy65yEMtdbRprQHT9KeUwpjRNpt6FLJMpPGJcxEUzshqpQrMGS0B_CRCLlmxqTWqBEPfpy3Y7aujfdqxmlVIkXqhghqlEzA3Ag==&sjdu=0ZFwD5rbjMRcHz87Kzx_g4iztdTb_2bOBnSpyL0eJqrnoH0zhNHGeUS0d52A-25mtSPp3ZGAbkfQnEhseHJ_YqiTRf1RGyCMrfm3GKjpLJXmRb1As35_i94VtIfO7AZtgI0QXqDAYweserYyqDu1JUW2c6S_FT2kfNZmCatNflk'

company_name <- read_html(test_url) %>%
    html_nodes(css= 'head > meta:nth-child(15)') %>%
    html_attr('content')

company_name

# /html/head/meta[12]
# /html/head/meta[13]
#html.js-focus-visible head meta
#/html/head/meta[13]
#head > meta:nth-child(15)
# head > meta:nth-child(14)

[1] "ABN AMRO Bank"

In [6]:
job_description

[1] "We zoeken een ware Einstein.Herken jij je hierin?Op het gebied van data ben jij een ware Einstein. Wat is er leuker dan data verwerken uit verschillende bronnen en die vervolgens omzetten naar bruikbare informatie? Wat jou betreft is het antwoord op die vraag: níets!Syncasso werkt hard aan het neerzetten van een state-of-the-art analytics landschap. We zijn in onze markt voorloper met +10 jaar ervaring in analytics en hebben een voorsprong door de beschikbaarheid van veel (verrijkte) data. We werken hard om onze organisatie nog meer data driven te maken door een top Analytics team neer te zetten. Wil jij met ons meedoen om deze ambitie te realiseren?Wat ga je bij ons doen? Als Data Scientist bij Syncasso heb je een centrale rol binnen onze organisatie. Je past Machine Learning toe en ontwikkelt betrouwbare voorspelmodellen en algoritmes om het incassoproces te optimaliseren. Hiervoor onderzoek je patronen in grote hoeveelheden data en past technieken toe om (on)gestructureerde data uit verschillende bronnen effectief te verzamelen en combineren. Je doorziet complexe vraagstellingen en vertaalt deze naar een oplosbaar optimalisatieprobleem. Bij het ontwikkelen van een oplossing houd je nauw contact met verschillende afdelingen en werk je samen met andere teams om analytische oplossingen te testen, implementeren en te monitoren in het proces. Waar nodig ga je de modellen updaten en finetunen. Je weet ook een brug te slaan naar mensen buiten je vakgebied. Je communiceert je inzichten proactief en begrijpelijk met interne en externe belanghebbenden op operationeel en strategisch niveau. Je blijft op de hoogte van de laatste ontwikkelingen binnen je vakgebied en kan die omzetten in de ontwikkeling van nieuwe oplossingen en innovaties voor Syncasso.Jouw sterke puntenJe hebt een Universitair werk- en denkniveau en een BSc of Msc in Artificial Intelligence, Data Science, Econometrie of Natuurwetenschap;Je hebt ervaring met Machine Learning, Datamining en Data Engineering technieken;Je hebt ervaring met statistische en analytische onderzoeksmethoden en valideert en verbetert van daaruit ML-oplossingen;Je hebt ervaring met relationele databases en het werken met openbare en ongestructureerde databronnen;Je beheerst SQL en een of meer van de volgende talen: R of Python;Plus: Ervaring met Azure Machine Learning, Jupyter Notebooks, DAX, PowerBI;Je vindt uitdagingen in het toepassen van Machine Learning technieken met financiële data;Je bent actief in zelfontwikkeling en wilt meegroeien in de ontwikkelingen.Wat we biedenJe krijgt veel ruimte om jezelf te ontwikkelen en impact te maken. Alle ruimte en kansen om te groeien en het beste uit jezelf te halen;De mogelijkheid om parttime (32 uur) te werken;Werken aan je ontwikkeling met e-learnings en workshops;Veel vrijheid. Thuiswerken of flexibele werktijden? Geen probleem;We helpen je mee je thuiswerkplek in te richten met o.a. een laptop, smartphone en extra beeldscherm;Een marktconform salaris met individueel keuzebudget om naar eigen inzicht te verdelen;Een uitstekende pensioenregeling;Een thuiswerkvergoeding per dag;8% vakantiegeld;Een collectieve zorgverzekering.CoronaproofSyncasso is Coronaproof. Bijna al onze medewerkers werken momenteel grotendeels thuis. Wij zijn ingericht om nieuwe collega’s op afstand te kunnen begeleiden en in te werken. We beschikken over de middelen en mogelijkheden om onze medewerkers met het oog op veiligheid, productiviteit en verbinding op een prettige manier hun werk te laten doen.Enthousiast?Solliciteer dan direct, je kunt snel een reactie verwachten.Vragen over Syncasso of deze functie? Neem dan contact op met Laura Boom, HR adviseur. Zij is bereikbaar op 06-51429857Als we een match zijn dan word je uitgenodigd voor een digitaal sollicitatiegesprek met de teamleider en één van onze HR adviseurs. Ook belangrijk om te weten: een geldige VOG en interne screening zijn onderdeel van de sollicitatieprocedure.Soort dienstverband: Fulltime, Parttime, Onbepaal

In [7]:
word_vec <- unname(unlist(sapply(job_description, function(z) str_split(tolower(z), " "))))
word_vec

[1] "we"                                 
  [2] "zoeken"                             
  [3] "een"                                
  [4] "ware"                               
  [5] "einstein.herken"                    
  [6] "jij"                                
  [7] "je"                                 
  [8] "hierin?op"                          
  [9] "het"                                
 [10] "gebied"                             
 [11] "van"                                
 [12] "data"                               
 [13] "ben"                                
 [14] "jij"                                
 [15] "een"                                
 [16] "ware"                               
 [17] "einstein."                          
 [18] "wat"                                
 [19] "is"                                 
 [20] "er"                                 
 [21] "leuker"                             
 [22] "dan"                                
 [23] "data"                               
 [24] "verwerken"                          
 [25] "uit"                                
 [26] "verschillende"                      
 [27] "bronnen"                            
 [28] "en"                                 
 [29] "die"                                
 [30] "vervolgens"                         
 [31] "omzetten"                           
 [32] "naar"                               
 [33] "bruikbare"                          
 [34] "informatie?"                        
 [35] "wat"                                
 [36] "jou"                                
 [37] "betreft"                            
 [38] "is"                                 
 [39] "het"                                
 [40] "antwoord"                           
 [41] "op"                                 
 [42] "die"                                
 [43] "vraag:"                             
 [44] "níets!syncasso"                     
 [45] "werkt"                              
 [46] "hard"                               
 [47] "aan"                                
 [48] "het"                                
 [49] "neerzetten"                         
 [50] "van"                                
 [51] "een"                                
 [52] "state-of-the-art"                   
 [53] "analytics"                          
 [54] "landschap."                         
 [55] "we"                                 
 [56] "zijn"                               
 [57] "in"                                 
 [58] "onze"                               
 [59] "markt"                              
 [60] "voorloper"                          
 [61] "met"                                
 [62] "+10"                                
 [63] "jaar"                               
 [64] "ervaring"                           
 [65] "in"                                 
 [66] "analytics"                          
 [67] "en"                                 
 [68] "hebben"                             
 [69] "een"                                
 [70] "voorsprong"                         
 [71] "door"                               
 [72] "de"                                 
 [73] "beschikbaarheid"                    
 [74] "van"                                
 [75] "veel"                               
 [76] "(verrijkte)"                        
 [77] "data."                              
 [78] "we"                                 
 [79] "werken"                             
 [80] "hard"                               
 [81] "om"                                 
 [82] "onze"                               
 [83] "organisatie"                        
 [84] "nog"                                
 [85] "meer"                               
 [86] "data"                               
 [87] "driven"                             
 [88] "te"                                 
 [89] "maken"                              
 [90] "door"                               
 [91] "een"                               

In [8]:
wordlist <- c("SQL", "Python", ".NET", "noSQL", "sneakers", 'jij')
test_table <- word_vec[word_vec %in% tolower(wordlist)]
test_table
unique(test_table)
data.frame(x = toString(unique(test_table)))


[1] "jij" "jij" "jij" "sql"

[1] "jij" "sql"

x
<chr>
"jij, sql"


# looping over multiple jobs

In [3]:
library(tidyverse)
library(rvest)

job_base_url <- 'https://nl.indeed.com'
job_base_search_url <- 'https://nl.indeed.com/jobs?q=data&start='
# url_job_search <- read_html('https://nl.indeed.com/jobs?q=data')

wordlist <- c("SQL", "Python", ".NET", "R", "Machine Learning")

jobs_df <- data.frame(
    Company = character(),
    job_title = character(),
    skills = character(),
    job_desc = character(),
    job_link = character()
)

for (i in seq.default(0, 20, 10)){

    job_list_url <- paste(job_base_search_url, as.character(i), sep= "")
    job_list_read <- read_html(job_list_url) # lees html in

    job_urls <- job_list_read %>%
    html_nodes(xpath= '//*[@id="mosaic-zone-jobcards"]') %>% # ga naar het specifieke xpath
    html_nodes('a') %>% # ga naar a
    html_attr("href")# pak de href

    # loop over job_urls
    for (j in 1:length(job_urls)){

        # when url is longer than 200 chars it is a job url
        if (nchar(job_urls[[j]]) > 200){
            full_job_url <- paste(job_base_url, job_urls[[j]], sep="")
            
            # read html
            job_html_read <- read_html(full_job_url)

            # extract company name
            company_name <- job_html_read %>%
                html_nodes(css= 'head > meta:nth-child(15)') %>%
                html_attr('content')

            # extract title
            job_title <- job_html_read %>%
                html_nodes('h1') %>%
                html_text()

            # extract description
            job_description <- job_html_read %>%
                html_nodes(xpath= '//*[@id="jobDescriptionText"]') %>%
                html_text()
            
            # generate wordvector
            ## hier moet nog een string split bij op speciale characters (.,?/;: etc.)
            wordvec <- unname(unlist(sapply(job_description, function(z) str_split(tolower(z), " "))))

            # extract skills from word vector
            skill_table <- wordvec[wordvec %in% tolower(wordlist)] %>%
                unique()
            
            jobs_df[nrow(jobs_df) + 1,] = c(
                company_name,
                job_title,
                toString(skill_table),
                job_description,
                full_job_url)
            
            Sys.sleep(4)
        }
    }
}

-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v dplyr   1.0.8
v tidyr   1.2.0     v stringr 1.4.0
v readr   2.1.2     v forcats 0.5.1

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


Attaching package: 'rvest'


The following object is masked from 'package:readr':

    guess_encoding




In [4]:
jobs_df

Company                                     
1  Syncasso                                    
2  Career Control                              
3  MN                                          
4  belastingdienst                             
5  Eiffel                                      
6  a.s.r.                                      
7  Eiffel                                      
8  Veneficus                                   
9  House of Bèta                               
10 Eiffel                                      
11 Veneficus                                   
12 Bertelsmann SE & Co. KGaA - Corporate Center
13 Veneficus                                   
14 Onrecruit voor TPS2                         
15 House of Bèta                               
16 YoungCapital                                
17 Grafton                                     
18 Veneficus                                   
19 Onrecruit voor TPS2                         
20 House of Bèta                               
   job_title                                              skills
1  Data Scientist                                         sql, r
2  Data Professional Talent Program                       sql   
3  Data Analist                                           sql   
4  Data-analist                                           r     
5  Data Analist                                           sql   
6  Informatie & Data Analist Risk & Beleggingen Dimension       
7  Data Traineeship                                       sql   
8  Junior data analist                                          
9  Traineeship Data Science                               sql   
10 Data Traineeship                                       sql   
11 Data Trainee                                                 
12 Data Science Graduate Program                          python
13 Junior data analist                                          
14 TRAINEESHIP BUSINESS IT & DATA                               
15 Traineeship Data Science                               sql   
16 Data verwerker                                               
17 Data analist Gemeente Amsterdam                              
18 Junior data analist                                          
19 TRAINEESHIP BUSINESS IT & DATA                               
20 Traineeship Data Science                               sql   
   job_desc                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   